In [48]:
import pandas as pd
import numpy as np
# from tqdm.notebook import tqdm
# from tqdm.autonotebook import tqdm
from tqdm import tqdm
from copy import deepcopy
# blah blah your code errored
tqdm._instances.clear()

In [49]:
macrotick = 100
sync_error = 0
time_out = 4 * 60 * 60
NUM_FLOW = 43
DATA_NAME = "single0"
TOPO_NAME = "0"

task = pd.read_csv("../../dac_data/%s.csv"%DATA_NAME)[:NUM_FLOW]
network = pd.read_csv("../../dac_data/%s_topology.csv"%TOPO_NAME)
for col in ['size','period','deadline','jitter']:
    task[col] = np.ceil(task[col] / macrotick).astype(int)
for col in ['t_proc','t_prop']:
    network[col] = np.ceil(network[col] / macrotick).astype(int)
    
nodes = list(network['link'].apply(lambda x:eval(x)[0])) + \
    list(network['link'].apply(lambda x:eval(x)[1]))
NODE_SET = list(set(nodes))
ES_set = [x for x in NODE_SET if nodes.count(x) == 2]
SW_set = list(set(NODE_SET) - set(ES_set))
LCM = np.lcm.reduce(task['period'])
net = np.zeros(shape = (max(NODE_SET) + 1, max(NODE_SET) + 1))

## 1. Model

In [50]:
net_var = {}

for _, row in network.iterrows():
    net_var.setdefault(row['link'], {})
    net_var[row['link']]['msd'] = row['t_proc']
    net[eval(row['link'])[0], eval(row['link'])[1]] = 1

In [51]:
## Create mapping from Link to index
link_to_index = {}
index_to_link = {}

counter = 0
for _, row in network.iterrows():
    link = row['link']
    link_to_index[link] = counter
    index_to_link[counter] = link
    counter += 1

Task model

In [52]:
## Shortest path
def bfs_paths(graph, start, goal):
    queue = [(start, [start])]
    while queue:
        (vertex, path) = queue.pop(0)
        for _next in set(np.reshape(np.argwhere(graph[vertex] > 0),  -1)) - set(path):
            if _next == goal:
                yield path + [_next]
            else:
                queue.append((_next, path + [_next]))

In [53]:
task_attr = {}
task_var = {}
for k, row in task.iterrows():
    task_attr.setdefault(k, {})
    task_var.setdefault(k, {})
    
    ## f_k
    task_attr[k]['t'] = int(row['src'])
    task_attr[k]['l'] = int(eval(row['dst'])[0])
#     task_attr[k]['ct'] = int(row['period']) 
#     task_attr[k]['rsl'] = int(row['size'] * 8) 
#     task_attr[k]['ml'] = int(row['deadline'])
    task_attr[k]['p'] = int(row['size'])
    task_attr[k]['T'] = int(row['period'])
    task_attr[k]['d'] = int(row['deadline'])
    task_attr[k]['L'] = task_attr[k]['p'] * 8
    
    task_attr[k]['route'] = next(bfs_paths(net, int(row['src']), int(eval(row['dst'])[0])))

In [54]:
route_to_index = {}
index_to_route = {}
for k in task_attr:
    route_to_index.setdefault(k, {})
    index_to_route.setdefault(k, {})
    for i, v in enumerate(task_attr[k]['route'][:-1]):
        route_to_index[k][str((v, task_attr[k]['route'][i+1]))] = i
        index_to_route[k][i] = str((v, task_attr[k]['route'][i+1]))

\begin{aligned}
&e s t_{i, t_{i}, t_{i}^{\prime}} \leftarrow r_{i}\\
&e s t_{i, a, b} \leftarrow e s t_{i, p r e c(a), a}+L_{i, p r e c(a), a}+d_{\text {prec }(a), a}+d_{a}+\delta\\
&l s t_{i, l_{i}^{\prime}, l_{i}} \leftarrow \tilde{d}_{i}-L_{i, l_{i}^{\prime}, l_{i}}-d_{l_{i}^{\prime}, l_{i}}\\
&l s t_{i, a, b} \leftarrow l s t_{i, b, s u c c(b)}-L_{i, a, b}-d_{a, b}-d_{b}-\delta
\end{aligned}

In [55]:
## It assumes a offset

est = {}
lst = {}

for k in task_attr:
    est.setdefault(k, [None] * (len(task_attr[k]['route']) - 1))
    lst.setdefault(k, [None] * (len(task_attr[k]['route']) - 1))
    for i in range(len(est[k])):
        if i == 0:
#             est[k][i] = np.random.randint(0, (task_attr[i]['T'] - task_attr[i]['d'] - task_attr[i]['L']) // 100) * 100
            est[k][i] = 0
            continue
        est[k][i] = est[k][i-1] + task_attr[k]['L'] + net_var[index_to_route[k][i]]['msd']
    
    for i in range(len(lst[k])-1, -1, -1):
        if i == len(lst[k])-1:
            lst[k][i] = est[k][0] + task_attr[k]['d'] - task_attr[k]['L'] - net_var[index_to_route[k][i]]['msd']
            continue
        lst[k][i] = lst[k][i+1] - task_attr[k]['L'] - net_var[index_to_route[k][i + 1]]['msd']
        
est_copy = deepcopy(est)
lst_copy = deepcopy(lst)

## 2. Tools

criterion values

$$\operatorname{crit}\left(f_{i, a, b}\right)=\frac{\tilde{d}_{i} \cdot\left(l s t_{i, a, b}-e s t_{i, a, b}\right)+\operatorname{order}\left(f_{i, a, b}\right)}{\operatorname{conflicts}\left(f_{i, a, b}\right)}$$

In [56]:
conflicts = {(k, link): 1 
               for k, link in[
             x for y in [[(x, str((f, task_attr[x]['route'][i+1])))
               for i,f in enumerate(task_attr[x]['route'][:-1])] 
               for x in task_attr]
               for x in y]}

def crit(task):
    global task_attr, lst_copy, est_copy, conflicts
    route_index = route_to_index[task[0]][task[1]]
    return (task_attr[task[0]]['d'] * (
        lst_copy[task[0]][route_index] - 
        est_copy[task[0]][route_index]
    ) + route_index) / conflicts[task]



## 3. EPIC

In [57]:
def getVar(uf):
    return sorted(uf, key = crit, reverse=False)[0]

In [58]:
def getBounds(k):
    global est, af, cs, assign
    task = af[k][0]
    current_link = af[k][1]
    current_index = route_to_index[task][current_link]
    
    if current_index > 0:
        last_ins = (task, index_to_route[task][current_index - 1])
        if last_ins in af:
            est[task][current_index] = assign[af.index(last_ins)][0] \
            + task_attr[task]['L'] + net_var[index_to_route[task][current_index]]['msd']
            cs[k].add(af.index(last_ins))
    return (est[task][current_index], 0)


**Frame isolation**

\begin{aligned}
&\forall \beta \in\left\{0, \ldots, \frac{\operatorname{lcm}\left\{T_{i}, T_{j}\right\}}{T_{j}}-1\right\}: \\
&\left(s_{i, a, b}+\alpha \cdot T_{i} \leq s_{j, y, a}+\beta \cdot T_{j}+L_{j, y, a}+d_{y, a}-\delta\right) \\
&\quad \vee\left(s_{j, a, b}+\beta \cdot T_{j} \leq s_{i, x, a}+\alpha \cdot T_{i}+L_{i, x, a}+d_{x, a}-\delta\right) \\
&\quad \vee\left(\lambda_{i, a, b} \neq \lambda_{j, a, b}\right)
\end{aligned}

**Overlap**

The wrong formular in the paper

\begin{aligned}
&s\left(a f\left[m^{\prime}\right]\right)-s(a f[k]) \% g \geq L(a f[k]) \\
&s(a f[k])-s\left(a f\left[m^{\prime}\right]\right) \% g \geq L\left(a f\left[m^{\prime}\right]\right)
\end{aligned}

The correct one from "Pira, Clément, and Christian Artigues. "Line search method for solving a non-preemptive strictly periodic scheduling problem." Journal of Scheduling 19.3 (2016): 227-243."

$$\left(t_{j}-t_{i}\right) \quad \bmod g_{i, j} \geq l_{i, j}, \quad \forall(i, j), i \neq j$$

In [59]:
def check(k, val):
    global assign, af, cs, gs
    val = list(val)
    task, link = af[k][0], af[k][1]

    success = True
    for k2, (task2, link2) in [(k2, (task2, link2))
                               for k2, (task2, link2) in enumerate(af)
                               if link2 == link and k2 != k]:
        if val[1] == assign[k2][1] and route_to_index[task][link] != 0:
            prec = index_to_route[task][route_to_index[task][link] - 1]
            prec2 = index_to_route[task2][route_to_index[task2][link2] - 1]
            if (task, prec) in af and (task2, prec2) in af:
                k_prec, k2_prec = af.index((task, prec)), af.index(
                    (task2, prec2))
                _lcm = int(np.lcm(task_attr[task]['T'], task_attr[task2]['T']))
                for a, b in [(a, b)
                             for a in range(_lcm // task_attr[task]['T'])
                             for b in range(_lcm // task_attr[task2]['T'])]:
                    frame_iso = \
                    val[0] + a * task_attr[task]['T'] < assign[k2_prec][0] + b * task_attr[task2]['T'] + net_var[link]['msd'] or\
                    assign[k2][0] + b * task_attr[task2]['T'] < assign[k_prec][0] + a * task_attr[task]['T'] + net_var[link2]['msd']
                    if frame_iso == False:
                        cs[k].add(k_prec)
                        cs[k].add(k2_prec)
                        if val[1] < 8 - 1:
                            val[1] += 1
                        else:
                            val[0] = np.inf
                        success = False
                        break
                    if not success:
                        break

        if success or val[0] <= lst[task][route_to_index[task][link]]:
            g = np.gcd(task_attr[task]['T'], task_attr[task2]['T'])
            d1 = (assign[k2][0] - val[0]) % g
            d2 = (val[0] - assign[k2][0]) % g
            if d1 < task_attr[task]['L']:
                cs[k].add(k2)
                val = (val[0] + task_attr[task2]['L'] + d1, 0)
                success = False
            elif d2 < task_attr[task2]['L']:
                cs[k].add(k2)
                val = (val[0] + task_attr[task2]['L'] - d2, 0)
                success = False
        if not success:
            return False, tuple(val)

    return True, (val[0], val[1] + 1) if val[1] < 8 - 1 else (val[0] + 1, val[1])

In [60]:
af = []
uf = [x for y in 
     [[(x, str((f, task_attr[x]['route'][i+1])))
       for i,f in enumerate(task_attr[x]['route'][:-1])] for x in task_attr]
     for x in y
     ]
allFrame = af + uf

assign = [None for i in range(len(allFrame))]
newVals = [(0,0) for k in range(len(allFrame))]

cs = [set() for i in range(len(allFrame))]
gs = set()

k = 0

In [61]:
while k < len(allFrame):
    if k == len(af):
        af.append(getVar(uf))
        uf.pop(uf.index(af[k]))
        val = getBounds(k)
    else:
        val = newVals[k]
    success = False
    while not success and val[0] <= lst[af[k][0]][route_to_index[af[k][0]][af[k][1]]]:
        assign[k] = val
        success, val = check(k, val)
    if success:
        newVals[k] = val
        k = k + 1
    else:
        if len(cs[k]) == 0 and len(gs) == 0:
            print('Infeasible')
            break
            
        ## Update criteria
        conflicts[af[k]] += len(cs[k])
        
        if gs and max(gs) > max(cs[k]):
            m = max(gs)
            gs = gs | cs[k] - set([m])
        else:
            m = max(cs[k])
            cs[m] = cs[m] | cs[k] - set([m])
        while k > m:
            assign[k] = None
            revert = af.pop(k)
            uf.append(revert)
            newVals[k] = (0,0)
            cs[k] = set()
            k = k - 1
        
       

## Output Schedule

In [62]:
## GCL
GCL = []
for i in range(len(af)):
    task, link = af[i]
    start, queue = assign[i]
    period = int(task_attr[task]['T'])
    # queue = i
    for t in range(int(LCM / period)):
        GCL.append([link, queue, 
        (start + t * period) * macrotick, 
        (start + task_attr[task]['L'] + t * period) * macrotick, 
        LCM * macrotick])

In [63]:
OFFSET = []
for i in task_attr:
    s_hop = str(tuple(task_attr[i]['route'][:2]))
    for index, (start, _) in enumerate(assign):
        if af[index][0] == i and af[index][1] == s_hop:
            OFFSET.append([i, 0, (task_attr[i]['T'] - start) * macrotick])

In [64]:
ROUTE = []
for i in task_attr:
    route = task_attr[i]['route']
    for h, v in enumerate(route[:-1]):
        ROUTE.append(
            [i, (v, route[h + 1])]
        )

In [65]:
QUEUE = []
for i in range(len(af)):
    task, link = af[i]
    _, queue = assign[i]
    # queue = i
    QUEUE.append([task, 0, eval(link), queue])
    # QUEUE.append([task, 0, eval(link), queue])

In [66]:
GCL = pd.DataFrame(GCL)
GCL.columns = ["link", "queue", "start", "end", "cycle"]
GCL.to_csv("COR2022-%s-%d-%s-GCL.csv"%(DATA_NAME,NUM_FLOW,TOPO_NAME), index=False)

OFFSET = pd.DataFrame(OFFSET)
OFFSET.columns = ['id', 'ins_id', 'offset']
OFFSET.to_csv("COR2022-%s-%d-%s-OFFSET.csv"%(DATA_NAME,NUM_FLOW,TOPO_NAME), index=False)

ROUTE = pd.DataFrame(ROUTE)
ROUTE.columns = ['id', 'link']
ROUTE.to_csv("COR2022-%s-%d-%s-ROUTE.csv"%(DATA_NAME,NUM_FLOW,TOPO_NAME), index=False)

QUEUE = pd.DataFrame(QUEUE)
QUEUE.columns = ['id','ins_id','link','queue']
QUEUE.to_csv("COR2022-%s-%d-%s-QUEUE.csv"%(DATA_NAME,NUM_FLOW,TOPO_NAME), index=False)